# Preliminary direct numerical simulation of 2D Rayleigh-Bénard convection

## Theoretical formulation
We consider the nondimensionalised equations in the Boussinesq approximation given by [Busse (1978)](https://doi.org/10.1088/0034-4885/41/12/003):
$$
\begin{align*}
    \mathrm{Pr}^{-1} \left[\partial_t \vec{u} + (\vec{u} \cdot \nabla) \vec{u}\right] &= - \nabla \pi + \nabla^2 \vec{u} + \theta \hat{z}, \\
    \partial_t \theta + (\vec{u} \cdot \nabla) \theta &= \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \nabla^2 \theta, \\
    \nabla \cdot \vec{u} &= 0
\end{align*}
$$
where:
- $\vec{u} = (u,0,w)$ is the dimensionless velocity,
- $\theta$ is the dimensionless temperature perturbation from the conductive (linear) base profile,
- $\pi$ is the dimensionless pressure perturbation from the hydrostatic base profile,
- $\mathrm{Pr}$ is the Prandtl number, the ratio of kinematic viscosity to thermal diffisivity,
- $\mathrm{Ra}$ is the Rayleigh number, the ratio of the thermal diffusion time scale to the convective time scale.

We impose no-slip, isothermal boundary conditions at the top and bottom of the domain,
$$
\begin{alignat*}{2}
    \vec{u}(z=0) &= \vec{u}(z=L_z) &&= \vec{0}, \\
    \theta(z=0) &= \theta(z=L_z) &&= 0,
\end{alignat*}
$$
and periodic lateral boundary conditions
$$
\begin{align*}
    \vec{u}(x=0) &= \vec{u}(x=L_x),\\
    \theta(x=0) &= \theta(x=L_x).
\end{align*}
$$

### Tau terms
Dedalus requires us to add *tau terms* of the form $\tau(x) P(z)$, where $\tau$ is a new degree of freedom and $P$ is a known polynomial, to the equations. This allows the non-periodic top and bottom boundary conditions to be imposed while ensuring that the system still has exact polynomial solutions (i.e., so it is solvable using spectral methods). Following [the steps in the documentation](https://dedalus-project.readthedocs.io/en/latest/pages/tau_method.html), we introduce six new degrees of freedom
$$
    \vec{\tau}_{u1}(x) = (\tau_1(x), 0, \tau_2(x)), \quad \vec{\tau}_{u2}(x) = (\tau_3(x), 0, \tau_4(x)),
    \quad \tau_{\theta 1}(x), \quad \text{and } \tau_{\theta 2}(x).
$$
Defining the auxiliary variables
$$
\begin{align*}
    G_u &= \nabla \otimes \vec{u} - \hat{z} \otimes \vec{\tau}_{u1}(x) P(z), \\
    \vec{G}_\theta &= \nabla \theta - \hat{z} \tau_{\theta 1}(x) P(z),
\end{align*}
$$
the equations become
$$
\begin{align*}
    \mathrm{Pr}^{-1} \partial_t \vec{u} - \nabla \cdot G_u + \nabla \pi - \theta \hat{z} + \vec{\tau}_{u2}(x) P(z) &= -\mathrm{Pr}^{-1} \vec{u} \cdot G_u, \\
    \partial_t \theta - \nabla \cdot \vec{G}_\theta - \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \tau_{\theta 2}(x) P(z) &= -\vec{u} \cdot \vec{G}_\theta, \\
    \operatorname{Tr}(G_u) &= 0.
\end{align*}
$$

### Pressure gauge condition
Since only the gradient of $\pi$ appears in the equations, there is a gauge freedom in the problem: the pressure is only determined up to a constant. Dedalus requires us to explicitly specify gauge conditions, so since $\pi$ is a pressure perturbation, we may as well require its mean value to be zero:
$$\int \pi ~\mathrm{d}V = 0.$$
Following [the documentation](https://dedalus-project.readthedocs.io/en/latest/pages/gauge_conditions.html), we must also introduce a seventh spatially constant tau variable $\tau_p$ to the divergence equation:
$$\operatorname{Tr}(G_u) + \tau_p = 0.$$


### Final PDE system
The final system of equations that we will enter into Dedalus is
$$
\begin{align*}
    \mathrm{Pr}^{-1} \partial_t \vec{u} - \nabla \cdot G_u + \nabla \pi - \theta \hat{z} + \vec{\tau}_{u2}(x) P(z) &= -\mathrm{Pr}^{-1} \vec{u} \cdot G_u, \\
    \partial_t \theta - \nabla \cdot \vec{G}_\theta - \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \tau_{\theta 2}(x) P(z) &= -\vec{u} \cdot \vec{G}_\theta, \\
    \operatorname{Tr}(G_u) + \tau_p = 0,
\end{align*}
$$
with the substitutions
$$
\begin{align*}
    G_u &= \nabla \otimes \vec{u} - \hat{z} \otimes \vec{\tau}_{u1}(x) P(z), \\
    \vec{G}_\theta &= \nabla \theta - \hat{z} \tau_{\theta 1}(x) P(z),
\end{align*}
$$
subject to the boundary conditions
$$
\begin{alignat*}{2}
    \vec{u}(z=0) &= \vec{u}(z=L_z) &&= \vec{0}, \\
    \theta(z=0) &= \theta(z=L_z) &&= 0
\end{alignat*}
$$
(the periodic lateral boundaries are handled automatically by using a Fourier basis in the horizontal).